In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 14 15:48:06 2023

@author: zijj
"""

#import scipy

import array
import control
import numpy as np
from control import *
import matplotlib.pyplot as plt


# open loop plant (idk if cool?)
g = 9.81
d = pass
L = pass
b = pass
m = pass
R = pass

num = [-5/7*g*(d/L)]
den = [1, (5*b)/(7*m*R), 0]

G = tf(num, den)

#G = tf([1], [1, 2]) #could be example of simple 1st order motor or RC low pass filter
G=tf(1, [1, 1])
#responsie output Y(s) op step input X(s)
X=tf([1],[1,0])   #step input 1/s
Y=G*X             #calculate output
print('input X = ',X)
print('transfer function = ',G)
print('output Y = ',Y)

# open loop motor

num = [K]
den = []

G2 = tf(num, den)

# open loop plant+motor

# closedloop everything


#open loop response = the system's output behavior without any feedback, measured after a step change in the systems input (= at the controller's output )
t,y = step_response(G,T=10,T_num=200, X0=0)     #X0 can be set as initial condition
t2,yc = impulse_response(G,T=10,T_num=200)
plt.plot(t,y, label='Step response - open loop')
plt.plot(t2,yc, label='Impulse response -  open loop')
plt.grid()
plt.xlabel('t [s]')
plt.ylabel('Vc [V]')
plt.legend()